# Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import regex as re
import emoji
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import seaborn as sns
import csv
from chardet import detect
from spellchecker import SpellChecker
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\casti\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\casti\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Loading all Datasets

In [3]:
dataset_pre = pd.read_csv('Datasets/PreElection.csv')

In [4]:
dataset_dr = pd.read_csv('Datasets/DuringElection.csv')

In [5]:
dataset_post = pd.read_csv('Datasets/PostElection.csv')

# Adding Election Period Labels by Adding Another Column

In [7]:
dataset_pre['Period'] = "PRE"


In [8]:
dataset_dr['Period'] = "DUR"

In [9]:
dataset_post['Period'] = "POST"

# Cleaning Datasets

In [10]:
#Concatenate all datasets
dataset = pd.concat([dataset_pre, dataset_dr, dataset_post], ignore_index=True)

In [12]:
dataset.shape

(16652, 4)

In [13]:
#1 Drop unnecessary columns

dataset.drop(['Date','Username'], axis=1, inplace=True)
dataset

,Tweet,Period
0,A year ago! Any progress from the winning cand...,PRE
1,"Bongbong Marcos, Sara Duterte express deepest ...",PRE
2,I decide I only vote for\rBBM❤️ SARA DUTERTE💚 ...,PRE
3,Mr. Bbm and Ms. Sara Duterte are obviously wil...,PRE
4,THE CROWD IS INSANE. BBM YOULL FOREVER BE INSA...,PRE
...,...,...
16647,Dictator Ferdinand Marcos from the Philippines...,POST
16648,This election rewrites history. People voted f...,POST
16649,"Hey, it’s a free country. No one has monopoly ...",POST
16650,HALA-lan. \n\n#Election2022PH,POST


In [14]:
#Get the exact duplicate tweets in the dataframe
print("Duplicate Tweets:", len(dataset['Tweet'])-len(dataset['Tweet'].drop_duplicates()))

Duplicate Tweets: 3108


In [15]:
dups = dataset["Tweet"]
dups_listed = dataset[dups.isin(dups[dups.duplicated()])].sort_values("Tweet")
print(dups_listed)

                                                   Tweet Period
3546     @biancadava\rLOOK: Members of the deaf commu...    PRE
4578     @biancadava\rLOOK: Members of the deaf commu...    PRE
3681     @mommydotsorry\r#Halalan2022 #UniteamBBMSara...    PRE
4212     @mommydotsorry\r#Halalan2022 #UniteamBBMSara...    PRE
3572     @nujp\rThis elections, never forget those wh...    PRE
...                                                  ...    ...
3790   🚨HALALAN 2022 ADVICE🚨\rplease! wag kayong papa...    PRE
15030  🤠Alrite let's discuss the Top 10-14 candidates...   POST
11318  🤠Alrite let's discuss the Top 10-14 candidates...   POST
110    🧵on 🇹🇼#Halalan2022| I’m sorry for failing y’al...    PRE
784    🧵on 🇹🇼#Halalan2022| I’m sorry for failing y’al...    PRE

[5440 rows x 2 columns]


In [16]:
# 2 Remove Exact Duplicate Tweets
# Drop the last duplicate and keep the first one
# Reset column index

dataset.drop_duplicates(subset='Tweet', keep="first", inplace=True)
dataset = dataset.reset_index()

In [17]:
# Drop old index column
dataset.drop(['index'], axis=1, inplace=True)
dataset

,Tweet,Period
0,A year ago! Any progress from the winning cand...,PRE
1,"Bongbong Marcos, Sara Duterte express deepest ...",PRE
2,I decide I only vote for\rBBM❤️ SARA DUTERTE💚 ...,PRE
3,Mr. Bbm and Ms. Sara Duterte are obviously wil...,PRE
4,THE CROWD IS INSANE. BBM YOULL FOREVER BE INSA...,PRE
...,...,...
13539,Dictator Ferdinand Marcos from the Philippines...,POST
13540,This election rewrites history. People voted f...,POST
13541,"Hey, it’s a free country. No one has monopoly ...",POST
13542,HALA-lan. \n\n#Election2022PH,POST


In [18]:
dataset['Tweet'] = dataset['Tweet'].str.replace("#", "hashtag")
dataset

,Tweet,Period
0,A year ago! Any progress from the winning cand...,PRE
1,"Bongbong Marcos, Sara Duterte express deepest ...",PRE
2,I decide I only vote for\rBBM❤️ SARA DUTERTE💚 ...,PRE
3,Mr. Bbm and Ms. Sara Duterte are obviously wil...,PRE
4,THE CROWD IS INSANE. BBM YOULL FOREVER BE INSA...,PRE
...,...,...
13539,Dictator Ferdinand Marcos from the Philippines...,POST
13540,This election rewrites history. People voted f...,POST
13541,"Hey, it’s a free country. No one has monopoly ...",POST
13542,HALA-lan. \n\nhashtagElection2022PH,POST


In [19]:
# 5 Remove non-alphanumeric characters, spaces & links
def clean_text(text):
    text = text.replace('?', ' ').replace('!', ' ') # Remove question marks, exclamation points
    text = ' '.join([word for word in text.split() if not word.startswith('@')]) # Remove mention tags
    text = text.replace('\n', ' ') # Remove newline characters

    # Remove all non-alphanumeric characters except hashtags, underscores, and spaces
    text = re.sub(r'[^a-zA-Z0-9_#\s]', '', text)

# Remove words containing "http" and "https"
    text = ' '.join([word for word in text.split() if not re.search(r'https', word)])
    text = ' '.join([word for word in text.split() if not re.search(r'http', word)])
    text = ' '.join([word for word in text.split() if not re.search(r'youtube', word)])
    text = ' '.join([word for word in text.split() if not re.search(r'vanBF1_BCyyo', word)])
    text = ' '.join([word for word in text.split() if not re.search(r'story_fbid', word)])
    text = ' '.join([word for word in text.split() if not re.search(r'listOLAK5uy', word)])
    text = ' '.join([word for word in text.split() if not re.search(r'edition_id', word)])
    text = ' '.join([word for word in text.split() if not re.search(r'News5GVGregorio', word)])
    text = ' '.join([word for word in text.split() if not re.search(r'utm_', word)])
    text = ' '.join([word for word in text.split() if not re.search(r'fbclid', word)])
    text = ' '.join([word for word in text.split() if not re.search(r'p23', word)])
    text = ' '.join([word for word in text.split() if not re.search(r'p24', word)])
    text = ' '.join([word for word in text.split() if not re.search(r'p25', word)])
    text = ' '.join([word for word in text.split() if not re.search(r'category_', word)])
    text = ' '.join([word for word in text.split() if not re.search(r'v_DUc7', word)])
    
    text = text.replace('...', ' ') # Remove ellipses
    text = ' '.join(text.split()) # Replace multiple spaces with a single space
    text = text.strip() # Remove leading/trailing spaces
    return text

dataset['Tweet'] = dataset['Tweet'].apply(clean_text)
print(dataset)

                                                   Tweet Period
0      A year ago Any progress from the winning candi...    PRE
1      Bongbong Marcos Sara Duterte express deepest g...    PRE
2      I decide I only vote for BBM SARA DUTERTE PULA...    PRE
3      Mr Bbm and Ms Sara Duterte are obviously will ...    PRE
4      THE CROWD IS INSANE BBM YOULL FOREVER BE INSAN...    PRE
...                                                  ...    ...
13539  Dictator Ferdinand Marcos from the Philippines...   POST
13540  This election rewrites history People voted fo...   POST
13541  Hey its a free country No one has monopoly of ...   POST
13542                      HALAlan hashtagElection2022PH   POST
13543  Cheating malalim na sugat kahit mahilom mahira...   POST

[13544 rows x 2 columns]


# Add Labels

In [22]:
def insert_label(row):
    period = row['Period']
    text = row['Tweet']

    if period == 'PRE':
        return 'PRE - ' + text
    elif period == 'DUR':
        return 'DUR - ' + text
    elif period == 'POST':
        return 'POST - ' + text
    
    return text

dataset['Tweet'] = dataset.apply(insert_label, axis=1)
dataset

,Tweet,Period
0,PRE - A year ago Any progress from the winning...,PRE
1,PRE - Bongbong Marcos Sara Duterte express dee...,PRE
2,PRE - I decide I only vote for BBM SARA DUTERT...,PRE
3,PRE - Mr Bbm and Ms Sara Duterte are obviously...,PRE
4,PRE - THE CROWD IS INSANE BBM YOULL FOREVER BE...,PRE
...,...,...
13539,POST - Dictator Ferdinand Marcos from the Phil...,POST
13540,POST - This election rewrites history People v...,POST
13541,POST - Hey its a free country No one has monop...,POST
13542,POST - HALAlan hashtagElection2022PH,POST


In [23]:
del dataset['Period']

In [24]:
dataset

,Tweet
0,PRE - A year ago Any progress from the winning...
1,PRE - Bongbong Marcos Sara Duterte express dee...
2,PRE - I decide I only vote for BBM SARA DUTERT...
3,PRE - Mr Bbm and Ms Sara Duterte are obviously...
4,PRE - THE CROWD IS INSANE BBM YOULL FOREVER BE...
...,...
13539,POST - Dictator Ferdinand Marcos from the Phil...
13540,POST - This election rewrites history People v...
13541,POST - Hey its a free country No one has monop...
13542,POST - HALAlan hashtagElection2022PH
